In [1]:
cd /data/projects/windturbine-identification/MachineLearningCourse

/data/projects/windturbine-identification/MachineLearningCourse


In [7]:
from scripts.windturbines.osm_download import getfeatures, get_param

import overpy

import fiona
from fiona.crs import from_epsg

from shapely.geometry import Point, mapping
import re

import argparse

import math
import time

from pathlib import Path

import os

import pandas as pd

In [8]:
def writeshape(nodes, name):

    schema = { 'geometry': 'Point', 'properties': {'method': 'str', 'source': 'str', 'elmw': 'float'} }
    shapeout = name

    with fiona.open(shapeout, 'w', crs=from_epsg(4326), driver='ESRI Shapefile', schema=schema) as output:
        for node in nodes:
            gensource = node.tags.get("generator:source", "unknown")
            genmethod = node.tags.get("generator:method", "unknown")
            genmodel = node.tags.get("generator:model", "unknown")

            genoutel = node.tags.get("generator:output:electricity", "unknown")
            genoutmw = -9999
            if re.search("[+-]?([0-9]*[.])?[0-9]+", genoutel):
                genoutmw = float(re.search("[+-]?([0-9]*[.])?[0-9]+", genoutel)[0])
                if "kw" in genoutel.lower():
                    genoutmw = genoutmw / 1000
            try:
                point = Point(node.lon, node.lat)
            except AttributeError:
                point = Point(node.center_lon, node.center_lat)
            prop = {
                'method': genmethod.lower(),
                'source': gensource.lower(),
                'elmw': genoutmw}
            output.write({'geometry': mapping(point), 'properties': prop})

In [10]:
# countries = ['IN', 'ES', 'GB', 'FR', 'BR', 'CA']

countries = ['FR', 'BR', 'CA']


for country in countries:
    
    print("Working on country: " + country)
    
    st = time.time()
    nodes, ways, rels = getfeatures(country, "power", "plant")
    features = nodes + ways + rels
    et = time.time()

    print('waiting for {:.0f} seconds'.format(et - st))
    time.sleep(et - st)

    print('fetching generators')
    nodes, ways, rels = getfeatures(country, "power", "generator")
    features = features + nodes + ways + rels

    PATH_OSM = get_param(country,"PATH_OSM")

    writeshape(features, PATH_OSM)



Working on country: FR
Starting download...
waiting for 10 seconds
waiting for 11 seconds
	nodes: 93, ways: 673, relations: 1056
waiting for 87 seconds
fetching generators
Starting download...
waiting for 36 seconds
waiting for 23 seconds


ERROR:fiona._env:data/osm/FR: Is a directory


	nodes: 7278, ways: 3721, relations: 17
Working on country: BR
Starting download...
waiting for 21 seconds
waiting for 6 seconds
	nodes: 10, ways: 201, relations: 63
waiting for 74 seconds
fetching generators
Starting download...
waiting for 16 seconds
waiting for 16 seconds


ERROR:fiona._env:data/osm/BR: Is a directory


	nodes: 10952, ways: 24285, relations: 2
Working on country: CA
Starting download...
waiting for 13 seconds
waiting for 11 seconds
	nodes: 5, ways: 295, relations: 42
waiting for 104 seconds
fetching generators
Starting download...
waiting for 56 seconds
waiting for 12 seconds


ERROR:fiona._env:data/osm/CA: Is a directory


	nodes: 5908, ways: 919, relations: 1


<function scripts.windturbines.osm_download.writeshape(nodes, name)>